In [1]:
#кластеризация, (k-mean), one hot encoding, автоматическая настройка параметров xgboost

import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn import *
import math as math 
from numpy.linalg import norm
%matplotlib inline
import matplotlib.pyplot as plt
import datetime

C:\Program Files (x86)\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Program Files (x86)\Anaconda3\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
C:\Program Files (x86)\Anaconda3\lib\site-packages\sklearn\lda.py:4: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will be removed in 0.19", DeprecationWarnin

In [ ]:
#Train
train = pd.read_csv('train.csv', delimiter=',', index_col='datetime', parse_dates=True)

train['year'] = train.index.year
train['month'] = train.index.month
train['hour'] = train.index.hour
train['day'] = train.index.weekday
train['calendar_day'] = train.index.day

season = train['month']
season = season.apply(lambda s: 4 if int(s) == 1 or int(s) == 2 else (int(s) - 3) // 3 + 1)   
train = train.drop('season', axis = 1)
train.insert(5, 'season', season)

hour = train['hour']
mainHour = np.arange(hour.shape[0])
mainHour = [1 if (hour[i] == 7) or (hour[i] == 8) or (hour[i] == 17) or (hour[i] == 18) else 0 for i in range(hour.shape[0])]
train['mainHour'] = mainHour

train.head(5)

In [ ]:
#Test
test = pd.read_csv('test.csv', delimiter=',', index_col='datetime', parse_dates=True)

test['year'] = test.index.year
test['month'] = test.index.month
test['hour'] = test.index.hour
test['day'] = test.index.weekday
test['calendar_day'] = test.index.day

season = test['month']
season = season.apply(lambda s: 4 if int(s) == 1 or int(s) == 2 else (int(s) - 3) // 3 + 1)   
test = test.drop('season', axis = 1)
test.insert(5, 'season', season)

hour = test['hour']
mainHour = np.arange(hour.shape[0])
mainHour = [1 if (hour[i] == 7) or (hour[i] == 8) or (hour[i] == 17) or (hour[i] == 18) else 0 for i in range(hour.shape[0])]
test['mainHour'] = mainHour


In [ ]:
# ONE_hot_encoding 
#for hours
hour_hot = pd.get_dummies(train['hour'], prefix='hour_hot')
train = train.join(hour_hot)
hour_hot_test = pd.get_dummies(test['hour'], prefix='hour_hot')
test = test.join(hour_hot_test)
#for weather
weather_hot = pd.get_dummies(train['weather'], prefix='weather_hot')
train = train.join(weather_hot)
weather_hot_test = pd.get_dummies(test['weather'], prefix='weather_hot')
test = test.join(weather_hot_test)
#for season
season_hot = pd.get_dummies(train['season'], prefix='season_hot')
train = train.join(season_hot)
season_hot_test = pd.get_dummies(test['season'], prefix='season_hot')
test = test.join(season_hot_test)

In [ ]:
train['hour2'] = (train['hour'])*2
test['hour2'] = (test['hour'])*2

train['dif1'] = (train['windspeed'] - train['atemp'])**2
test['dif1'] = (test['windspeed'] - test['atemp'])**2

# train['hum_div_seas'] = (train['humidity']/(train['day']))*2
# test['hum_div_seas'] = (test['humidity']/(test['day']))*2

# train['hum_div_seas'] = (train['temp']/(train['humidity']))*2
# test['hum_div_seas'] = (test['temp']/(test['humidity']))*2



#_______________________________________________________________________________________________________________________________
#562
# param = 5

# train_no_count = train.drop(['count', 'casual', 'registered'], axis = 1)
# ten_per_train = train_no_count[0:1000]
# kmeans = cluster.KMeans(n_clusters=param, random_state=0).fit(ten_per_train)

# labels = kmeans.predict(train_no_count)
# train['labels'] = labels

# labels = kmeans.predict(test)
# test['labels'] = labels

# #for cluster labels 
# labels_hot = pd.get_dummies(train['labels'], prefix='labels_hot')
# train = train.join(labels_hot)

# labels_hot_test = pd.get_dummies(test['labels'], prefix='labels_hot')
# test = test.join(labels_hot_test)

X = train.drop(['mainHour','casual', 'registered', 'count'], axis = 1)
Xr = train.drop(['casual', 'registered', 'count'], axis = 1)
X_test = test.drop(['mainHour'], axis = 1)
Xr_test = test

# _______________________________________________________________________________________________________________________________

# kmeans = cluster.KMeans(n_clusters=param, random_state=0).fit(X)
# X['labels'] = kmeans.labels_
# kmeans = cluster.KMeans(n_clusters=param, random_state=0).fit(Xr)
# Xr['labels'] = kmeans.labels_
# kmeans = cluster.KMeans(n_clusters=param, random_state=0).fit(X_test)
# X_test['labels'] = kmeans.labels_
# kmeans = cluster.KMeans(n_clusters=param, random_state=0).fit(Xr_test)
# Xr_test['labels'] = kmeans.labels_

#564
# ten_per_X = X[0:1000]
# kmeans = cluster.KMeans(n_clusters=param, random_state=0).fit(ten_per_X)
# labels = kmeans.predict(X)
# X['labels'] = labels

# ten_per_Xr = Xr[0:1000]
# kmeans = cluster.KMeans(n_clusters=param, random_state=0).fit(ten_per_Xr)
# labels = kmeans.predict(Xr)
# Xr['labels'] = labels

# ten_per_X_test = X_test[0:1000]
# kmeans = cluster.KMeans(n_clusters=param, random_state=0).fit(ten_per_X)
# labels = kmeans.predict(X_test)
# X_test['labels'] = labels

# ten_per_Xr_test = Xr_test[0:1000]
# kmeans = cluster.KMeans(n_clusters=param, random_state=0).fit(ten_per_Xr)
# labels = kmeans.predict(Xr_test)
# Xr_test['labels'] = labels


In [ ]:
maxDepth = 3
# nEstimators = 700
# nEstimators = 900
nEstimators = 1200
subSample = 0.9

y = np.log(train['casual'] + 1)
xgb_model = xgb.XGBRegressor(max_depth = maxDepth, n_estimators = nEstimators, subsample = subSample, nthread = 10).fit(X,y)
# xgb_model = calibration.CalibratedClassifierCV(xgb_model, method='sigmoid', cv=3).fit(X,y)
predictions1 = np.exp(xgb_model.predict(X)) - 1

# print(predictions1)
# aucXGBTrain = metrics.roc_auc_score(y, predictions1)
# print('XGBoost:       {:6.4}'.format(aucXGBTrain))
#-----------------------------------
y = np.log(train['registered'] + 1)
xgb_model = xgb.XGBRegressor(max_depth = maxDepth, n_estimators = nEstimators, subsample = subSample, nthread = 10).fit(Xr,y)
# xgb_model = calibration.CalibratedClassifierCV(xgb_model, method='sigmoid', cv=3).fit(X,y)
predictions2 = np.exp(xgb_model.predict(Xr)) - 1
#-----------------------------------
y = train['count']
predictions = predictions1 + predictions2
predictions = np.round(predictions)

print(min(abs(predictions - y)))
print(math.sqrt(1 / len(y) * sum((np.log(predictions + 1) - np.log(y + 1))**2)))
print(max(abs(predictions - y)))
print(len([elem for elem in (abs(predictions - y) / y) if elem < 0.1]), train.shape[0])

In [ ]:
y = np.log(train['casual'] + 1)

xgb_model = xgb.XGBRegressor(max_depth = maxDepth, n_estimators = nEstimators, subsample = subSample, nthread = 8).fit(X,y)
predictions1 = np.exp(xgb_model.predict(X_test)) - 1
#--------------------------
y = np.log(train['registered'] + 1)

xgb_model = xgb.XGBRegressor(max_depth = maxDepth, n_estimators = nEstimators, subsample = subSample, nthread = 8).fit(Xr,y)
predictions2 = np.exp(xgb_model.predict(Xr_test)) - 1
#--------------------------
predictions = predictions1 + predictions2
predictions = np.round(predictions)
predictions = [0 if predictions[i] < 0.01 else predictions[i] for i in range(predictions.shape[0])]

test.insert(len(test.columns), 'count', predictions)
test.head(5)

In [ ]:
xgb.plot_importance(xgb_model)

In [ ]:
newny = pd.DataFrame(test.index)
newny.insert(1, 'count', predictions)
newny.head(20)

newny.to_csv('submission.csv', sep=',', header = True, index = False)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
fig, ax = plt.subplots()

x1 = train.index
x2 = test.index

y1 = train['count']
y2 = test['count']

plt.plot(x1, y1, 'g.', x2, y2, 'r.')
plt.show()

In [ ]:
# x1 = train.index
# x2 = test.index

x1 = train['temp']
x2 = test['temp']

y1 = train['count']
y2 = test['count']

plt.plot(x1, y1, 'g*', x2, y2, 'r.')
plt.show()

In [ ]:
x1 = train['humidity']
x2 = test['humidity']

y1 = train['count']
y2 = test['count']

plt.plot(x1, y1, 'g*', x2, y2, 'r.')
plt.show()

In [ ]:
x2 = test.index
y2 = test['count']

plt.plot(x2, y2, 'r.')
plt.show()